In [3]:
import pandas as pd

# Cargar los archivos CSV
biogrid_data = pd.read_csv('Biogrid_data_propionibacterium_freudenreichii.csv')
intact_data = pd.read_csv('Intact_data_Propionibacterium_freudenreichii.csv')
string_data = pd.read_csv('String_data_Propionibacterium_freudenreichii.csv')
deseq_data = pd.read_csv('deSeqRes_combined_with_GO_Biological_Process_terms_sorted.csv')

# Renombrar las columnas del archivo de Biogrid
biogrid_data = biogrid_data.rename(columns={
    "Official symbol for Interactor A": "Interactor A Genes",
    "Official symbol for Interactor B": "Interactor B Genes"
})

# Renombrar las columnas del archivo de String
biogrid_data = string_data.rename(columns={
    "Organism ID for Interactor A": "Taxid interactor A",
    "Organism ID for Interactor B": "Taxid interactor B"
})

# Seleccionar solo las columnas de interés y eliminar las columnas especificadas
biogrid_data = biogrid_data[["Interactor A Genes", "Interactor B Genes", "Taxid interactor A", "Taxid interactor B", "Genes"]]
intact_data = intact_data[["Interactor A Genes", "Interactor B Genes", "Taxid interactor A", "Taxid interactor B", "Genes"]]
string_data = string_data[["Interactor A Genes", "Interactor B Genes", "Taxid interactor A", "Taxid interactor B", "Genes"]]

# Realizar el merge de los archivos en un solo DataFrame
merged_data = pd.merge(biogrid_data, intact_data, on=['Genes', 'Interactor A Genes', 'Interactor B Genes', 'Taxid interactor A', 'Taxid interactor B'], how='outer')
merged_data = pd.merge(merged_data, string_data, on=['Genes', 'Interactor A Genes', 'Interactor B Genes', 'Taxid interactor A', 'Taxid interactor B'], how='outer')

# Eliminar duplicados
merged_data = merged_data.drop_duplicates()

# Lista de genes para filtrar
# Lista de genes para filtrar
genes_to_filter = [
    "SH2D3C", "SHEP1", "CORO1A", "ARID5A", "KIFC2", "OASL", "KIF20A", "B3GALT4"
]

# Filtrar el DataFrame por los genes especificados
filtered_data = merged_data[merged_data['Genes'].isin(genes_to_filter)]

# Mostrar las primeras filas del DataFrame filtrado para verificar
print(filtered_data.head())

# Manejar la información de deseq_data por separado
deseq_data_filtered = deseq_data[deseq_data['Genes'].isin(genes_to_filter)]

# Mostrar las primeras filas del DataFrame de deseq_data filtrado para verificar
print(deseq_data_filtered.head())

    Interactor A Genes Interactor B Genes Taxid interactor A  \
841                TTK             KIF20A       Homo sapiens   
842             KIF20A              AURKA       Homo sapiens   
843             INCENP             KIF20A       Homo sapiens   
844             KIF20A             INCENP       Homo sapiens   
845             KIF20A               HMMR       Homo sapiens   

    Taxid interactor B   Genes  
841       Homo sapiens  KIF20A  
842       Homo sapiens  KIF20A  
843       Homo sapiens  KIF20A  
844       Homo sapiens  KIF20A  
845       Homo sapiens  KIF20A  
Empty DataFrame
Columns: [Genes, Term, Adjusted P-value, Log2FoldChange, FoldChange, Regulation]
Index: []


In [4]:
import pandas as pd

# Cargar los archivos CSV
#filtered_data = pd.read_csv('filtered_data.csv')  # Asegúrate de tener este archivo generado previamente
data_with_fc = pd.read_csv('deSeqRes_filtrado_Propionibacterium freudenreichii_with_FC.csv')
data_with_terms = pd.read_csv('deSeqRes_combined_with_GO_Biological_Process_terms_sorted.csv')

# Renombrar las columnas del archivo de Fold Change
data_with_fc = data_with_fc.rename(columns={
    'GeneSymbol': 'Genes',
    'DirectionalFoldChange': 'FoldChange'
})

# Crear una nueva columna 'Regulation' basada en el signo de 'FoldChange'
data_with_fc['Regulation'] = data_with_fc['FoldChange'].apply(lambda x: 'UpRegulated' if x > 0 else 'DownRegulated')

# Tomar el valor absoluto de 'FoldChange'
data_with_fc['FoldChange'] = data_with_fc['FoldChange'].abs()

# Lista de genes para filtrar
# Lista de genes para filtrar
genes_to_filter = [
    "SH2D3C", "SHEP1", "CORO1A", "ARID5A", "KIFC2", "OASL", "KIF20A", "B3GALT4"
]

# Filtrar data_with_fc por los genes especificados
data_with_fc_filtered = data_with_fc[data_with_fc['Genes'].isin(genes_to_filter)]

# Filtrar data_with_terms por los genes especificados
data_with_terms_filtered = data_with_terms[data_with_terms['Genes'].isin(genes_to_filter)]

# Crear listas para nodos y aristas
nodos = set()
aristas = []

# Iterar sobre cada fila del DataFrame filtered_data para crear nodos y aristas
for index, row in filtered_data.iterrows():
    gene = row['Genes']
    if gene in genes_to_filter:
        nodos.add((gene, 'Gene'))  # Agregar el gen como nodo

        # Agregar interacciones como aristas
        interactor_a = row['Interactor A Genes']
        interactor_b = row['Interactor B Genes']
        taxid_a = row['Taxid interactor A']
        taxid_b = row['Taxid interactor B']
        if pd.notna(interactor_a):
            aristas.append((gene, interactor_a, 'pp', taxid_a, taxid_b))
            nodos.add((interactor_a, 'Gene'))  # Agregar interactor A como nodo
        if pd.notna(interactor_b):
            aristas.append((gene, interactor_b, 'pp', taxid_a, taxid_b))
            nodos.add((interactor_b, 'Gene'))  # Agregar interactor B como nodo

# Asegurarse de que todos los genes en genes_to_filter estén en nodos
for gene in genes_to_filter:
    nodos.add((gene, 'Gene'))

# Crear DataFrame de nodos
nodos_df = pd.DataFrame(list(nodos), columns=['Id', 'Type'])

# Realizar el merge para completar la información faltante con data_with_fc_filtered
nodos_df = pd.merge(nodos_df, data_with_fc_filtered[['Genes', 'FoldChange', 'Regulation']], left_on='Id', right_on='Genes', how='left')

# Realizar el merge para agregar la información de términos con data_with_terms_filtered
nodos_df = pd.merge(nodos_df, data_with_terms_filtered[['Genes', 'Term']], left_on='Id', right_on='Genes', how='left')

# Eliminar la columna 'Genes' después del merge, si existe
if 'Genes' in nodos_df.columns:
    nodos_df = nodos_df.drop(columns=['Genes'])

# Eliminar duplicados en nodos_df
nodos_df = nodos_df.drop_duplicates()

# Filtrar el DataFrame de nodos por los genes especificados
nodos_df = nodos_df[nodos_df['Id'].isin(genes_to_filter)]

# Eliminar las columnas Genes_x y Genes_y si existen
columns_to_drop = ['Genes_x', 'Genes_y']
nodos_df = nodos_df.drop(columns=[col for col in columns_to_drop if col in nodos_df.columns])

# Guardar el archivo de nodos actualizado
nodos_df.to_csv('nodos.csv', index=False)

# Crear DataFrame de aristas con información adicional
aristas_df = pd.DataFrame(aristas, columns=['Source', 'Target', 'Interaction', 'Taxid interactor A', 'Taxid interactor B'])

# Eliminar duplicados en aristas
aristas_df = aristas_df.drop_duplicates()

# Guardar el archivo de aristas
aristas_df.to_csv('aristas.csv', index=False)

# Mostrar las primeras filas del DataFrame de nodos actualizado para verificar
print(nodos_df.head())

# Mostrar las primeras filas del DataFrame de aristas actualizado para verificar
print(aristas_df.head())

         Id  Type  FoldChange     Regulation Term
1     KIFC2  Gene    4.421851    UpRegulated  NaN
2    KIF20A  Gene    4.588599  DownRegulated  NaN
12   SH2D3C  Gene  111.764635    UpRegulated  NaN
13   CORO1A  Gene   13.869777    UpRegulated  NaN
18  B3GALT4  Gene    9.558074  DownRegulated  NaN
   Source  Target Interaction Taxid interactor A Taxid interactor B
0  KIF20A     TTK          pp       Homo sapiens       Homo sapiens
1  KIF20A  KIF20A          pp       Homo sapiens       Homo sapiens
3  KIF20A   AURKA          pp       Homo sapiens       Homo sapiens
4  KIF20A  INCENP          pp       Homo sapiens       Homo sapiens
9  KIF20A    HMMR          pp       Homo sapiens       Homo sapiens
